# Tutorial 2 - Bulk RNA-Seq DEG Analysis for Mouse

## Overview

This tutorial demonstrates how to use read count data, here generated from [Tutorial 1b](https://github.com/King-Laboratory/scRNASeq-miRNASeq-and-TF-Network-Analysis/blob/bda75860ace82cf180a6f9eae115ebaf2eabc5f9/Bulk_RNA-Seq_Tutorials/Bulk_RNA-Seq_Mouse/Tutorial_1B_alignment_full_dataset_mouse.ipynb) to generate a list of differentially expressed genes. Running the code in this tutorial should take approximately 15 minutes.

This workflow uses the R DESeq2 package, with additional documentation found here: https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html

![Mouse Bulk RNA-seq workflow](../../images/count-workflow.png)

## Learning Objectives

- Explore an example Bulk RNA-sequencing dataset
- Understand the workflow of using read counts to generate a list of differentially expressed genes (DEG), including:
    - R package install and loading
    - Experimental design
    - Data normalization
    - Plotting
    - Contrast comparisons
- Annotate and export list of DEGs

## STEP 1: Getting Started

<div class="alert alert-block alert-warning"> NOTE: This Jupyter Notebook was developed to run within a customized container on AWS with all software and packages pre-configured. If running without this customized container, you will need to install tools using the Miniforge environment setup instructions below before moving on to Step 2.</div>

### Without Container: Install Miniforge and R Packages

Miniforge is a lightweight Conda distribution that offers a streamlined installation process and efficient package management. It provides access to a vast repository of packages.

The following code performs these steps:
- Downloads Miniforge or Mambaforge (you can use either based on preference)
- Installs Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
- Installs gsutil and dependencies
- Using miniforge and bioconda, installs R packages that will be used in this tutorial


<div class="alert alert-block alert-info">Tip: If using the Miniforge install, run the following code cells by removing the # pound from each command line. </div>

In [ ]:
#system('curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh', intern = TRUE)

#system('bash Miniforge3-$(uname)-$(uname -m).sh -b -u -p $HOME/miniforge', intern = TRUE)

#Sys.setenv(PATH = paste(Sys.getenv("HOME"), "/miniforge/bin:", Sys.getenv("PATH"), sep = ""))

In [ ]:
#system('mamba install -y -c conda-forge -c bioconda gsutil', intern = TRUE)

### STEP 1: Library Installation

<div class="alert alert-block alert-info">Tip: If using the Miniforge install, run the following code cells by removing the # pound from each command line. </div>

In [ ]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#options(repos = BiocManager::repositories())

#BiocManager::install(c("ComplexHeatmap", "DESeq2", "EnhancedVolcano"), force = TRUE)

#install.packages(c("dplyr", "pheatmap", "ggrepel"), dependencies = TRUE)


----------------------------------------------------

## If running from a container, as noted above, start with <b> STEP 2 </b> below:

### STEP 2: Load Libraries

In [ ]:
# Install CRAN package ggfortify, even if running from container
install.packages("ggfortify", dependencies = TRUE)

In [ ]:
# Load libraries
library(DESeq2)
library(dplyr)
library(ComplexHeatmap)
library(EnhancedVolcano)
library(ggplot2)
library(ggrepel)
library(ggfortify)

### STEP 3: Reading in Data

This step retrieves the merged read counts data from all samples of Bioproject PRJNA892075, and the feature table, from an AWS S3 Bucket. This step involves reformatting or adjusting the data.

To make things easier, we created a matrix, or a table, of just our read-count data -- excluding the non-numerical metadata columns. This makes it easy for us to feed this table into the DESeq2 tool later on.

Additionally, we will later use normalization involving logarithms. So, to prevent errors, we will replace all read count values of '0' in our data, with '1'. This will change the data only slightly, and will prevent these zeros from causing an 'undefined' or 'N/A' math error in the logarithm normalization step.


In [ ]:
# Check if the directories exist before creating them
if (!dir.exists("data/gene_counts")) {
  dir.create("data/gene_counts", recursive = TRUE)
}

if (!dir.exists("data/reference")) {
  dir.create("data/reference", recursive = TRUE)
}

# Path to the gene counts file
!system("wget -P data/gene_counts/ https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/readcounts/merged_gene_counts.txt")
!system("wget -P data/reference/ https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/reference/mouse_feature_table.txt")

counts_file <- "data/gene_counts/merged_gene_counts.txt"
feature_table<- "data/reference/mouse_feature_table.txt"

In [ ]:
#read-in the raw gene count file to a dataframe variable we named 'read_counts'
read_counts <- read.table('data/gene_counts/merged_gene_counts.txt',head=TRUE)

#amount of columns the dataframe has
cols = ncol(read_counts)

#assign the numerical read counts to a matrix variable we named 'rnaseqMatrix'
rnaseqMatrix <- round(read_counts[,c(2:cols)])

#label the rownames of this matrix with the rowname column from the gene count dataframe.
rownames(rnaseqMatrix) <- read_counts[,1]

head(rnaseqMatrix)

### STEP 4: Specifying Experimental Design

Next, we will specify the experimental design. 

The DESeq2 package will later use the experimental design to group samples together, and output information about the statistical differences in gene expression between these specified groups.

In [ ]:
# define the sample experimental design, in this case 3 wild-type and 3 bacteriophage infected samples
samples_treatment <- data.frame(matrix(c(rep("pAd PSAP",2),rep("pAd GFP",2)),ncol=1))
samples_ID <- data.frame(matrix(colnames(rnaseqMatrix),ncol=1))
samples <- cbind(samples_ID,samples_treatment)
names(samples) <- c("ID","Treatment")
rownames(samples) <- samples[,1]

print("An example of how a DESeq2 experimental design table might look.")
samples



### STEP 5: Creating DESeq2 Object

Now use the treatment design matrix in combination with the read count matrix to create a DESeq2 object. 

Once created, this is also a good opportunity to filter out lowly expresseed genes, and to inspect the pre-normalized data using pairwise comparison plots. These example plots, the pairwise plot and boxplot, can be used to examine gene expression distributions and highlight outliers.

Finally, the DESeq2 analysis can be run on the DESeq2 object.

In [ ]:
#Create the DESeq2 DataSet object
deseq2Data <- DESeqDataSetFromMatrix(countData = rnaseqMatrix, colData = samples, design = ~ Treatment)

# Perform pre-filtering of the data (DESeq2 also automatically does this, but this can increase speed)
deseq2Data <- deseq2Data[rowSums(counts(deseq2Data)) > 10, ]

# Inspect pre-normalized data 
pairs(log2((counts(deseq2Data)+1)))
boxplot(log2((counts(deseq2Data)+1)))

# Run pipeline for differential expression steps
deseq2Data <- DESeq(deseq2Data)


### STEP 6: Normalize Data

DESeq2 automatically normalizes data in the DESeq() step using a method called median of ratios. For our own downstream analysis, we can pick from several different normalization methods. As an example, here we use rlog transformation, or regularized log transformation to normalize data. This method is similar to log normalization, but includes a 'shrinkage' estimator to attempt to shrink the variance of the log-transformed data. More information for this and other normalization methods can be found in the DESeq2 documentation. 

Normalization of gene read counts across samples will help to minimize sample variation noise and ensure more accurate comparisons. Looking at normalized plots can be an easy way to look out for any obvious inconsistencies or errors in the sample data. 

In [ ]:
#perform rlog normalization on DESeq2 object
rld <- rlog(deseq2Data, blind=FALSE)

#a handy variable so we can easily reference the rlog genecount matrix in the future, as DESeq objects contain many tables of data.
rlogcounts <- assay(rld)
rownames(rlogcounts) <- rownames(deseq2Data)

head(rlogcounts)

Normalization of data can is often verified by inspecting by various plots, such as histograms (these typically follow a negative binomial distribution, as shown below):

In [ ]:
hist(rlogcounts)

Boxplots:

In [ ]:
boxplot(rlogcounts)

Or pairwise comparison plots. Notice how sample groups can be visualized just by looking at the pairwise plot.

In [ ]:
pairs(rlogcounts)

Compare the plots of the pre-normalized and normalized data to see the effect of normalization.

Notice how the samples in the box plot, and in-group samples in the pair plots display similar distributions.

Looking at normalized plots can be an easy way to look out for any obvious inconsistencies or errors in the sample data.

### STEP 7: PCA Plot

We can now begin to produce plots to further analyze the differences between groups.

PCA plots can be an effective way to visualize variation within and between sample sets.

In [ ]:
pca_counts <- prcomp(t(rlogcounts))
autoplot(pca_counts, data = samples, colour="Treatment", size=6)

Plots can be output as above, or also saved as pdf or image files, as below. Try to navigate and click to the created file, after you run the code. 

Tiff files can also be created this way.

In [ ]:
pdf("data/gene_counts/PCA_Plot.pdf")
autoplot(pca_counts, data = samples, colour="Treatment", size=6)
dev.off()

tiff("data/gene_counts/PCA_Plot.tiff")
autoplot(pca_counts, data = samples, colour="Treatment", size=6)
dev.off()    

### STEP 8: Contrast Comparisons

Contrast comparisons between two treatment groups can be performed using the DESeq2 package to identify differentially up and down regulated genes.

Using R, these results can be summarized, reordered, and/or trimmed.

Differentially expressed genes are typically quantified as those genes which have an adjusted P value of below 0.05. An adjusted P value can also be referred to as a false discovery rate, or FDR. In the below, there are 1,357 differentially expressed genes. For the sake of reducing the amount of differentially expressed genes, to make analyses easier, these could also be further refined by introducing a threshold based on fold-change. DESeq2 uses the Wald and Benjamini-Hochberg tests for determining P and Adjusted P-values, respectively. More information on this can be found in the DESeq2 documentation. 

In [ ]:
#Output the results of comparing WT and BPs_lysogen group
res_PSAP_GFP <- results(deseq2Data, contrast=c("Treatment","pAd PSAP","pAd GFP"))
summary(res_PSAP_GFP)
print("Number of genes under a p value of 0.05")
sum(res_PSAP_GFP$pvalue < 0.05, na.rm=TRUE)
print("Number of genes under a p adjusted value of 0.05")
sum(res_PSAP_GFP$padj < 0.05, na.rm=TRUE)


#DESeq2 optionally output some padj values as 'NA' when gene counts meet certain criteria
#that criteria can be found here: https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA
#for practical uses, one could replace the NA values with '1', essentially NA values not significant.
res_PSAP_GFP$padj[is.na(res_PSAP_GFP$padj)] <- 1

### STEP 9: Annotating and Exporting Results

Differentially expressed genes can be annotated and exported. Combining gene counts with gene annotation is not always simple, and will vary heavily depending on the annotation file itself.

For example, when merging gene counts by 'gene id' with feature tables from NCBI's assembly database, it is very common to have duplicate rows, as a single 'gene id' may match to multiple features. 

Depending on how you want to use your annotated results, you may decide to further reformat your table to better suit your further analysis, for instance by downloading and using a spreadsheet editor.

Because the transcript table for mouse we have here is simple, it is easy for us to reformat it in the way we want just using R code. However, it is important to note that, again, this is very often not the case, and reformatting will vary depending on both the annotation file itself, and what that annotation file will be used for.

In [ ]:
#read-in the annotation file (see extended tutorial for more info on how to get annotation files) and store it into a variable.
annotation <- read.table("data/reference/mouse_feature_table.txt", header=TRUE,sep='\t', quote="\"",comment.char = "")
#put the previous DESeq results into an easier to manipulate data.frame format
results <- data.frame(res_PSAP_GFP)
#merge the annotation with the DESeq2 contrast comparison table
results.annot <- merge(results,annotation,by.x=0,by.y=17,all.x=TRUE)

#Open the annotation file above to see the format. If you further download and examine in a spreadsheet editor,
#you can see there is one CDS or ..RNA identifier for each 'gene' feature
#this makes reformatting in R simple by just removing every 'gene' feature, and leaving the more detailed 'cds' or '...RNA' feature.
#this is very often not always the case, but simple and easy here.
results.annot <- results.annot[annotation$X..feature!='gene',]

#labeling the columns properly after the merge
colnames(results.annot)[1] <- 'locus_tag'

#write out the file
write.table(results.annot,"data/gene_counts/PSAP_GFP_DESeq2_annot.txt",row.names=FALSE,sep="\t")

head(results.annot)


### STEP 10: More Plots and Clustering Data

Finally, although several different kind of analysis can be done, it is common to plot the results of differential gene expression analysis in visualizations such as volcano plot, MA plot, and heatmaps. These plots help one to further examine the output list of differentially expressed genes in different ways, and potentially identify or further investigate candidate gene(s).

In [ ]:
#Volcano and MA plots can be used to visualize significant differences in gene expression
results_volc_df <- results.annot
EnhancedVolcano(results_volc_df,
    lab = results_volc_df[,1],
    x = 'log2FoldChange',
    y = 'padj')

#ma plot
plotMA(res_PSAP_GFP)


#heatmaps can used to look at clustering and expression of various gene lists.
filtered_res <- results.annot %>% filter(padj < 0.05)
filtered_res_counts <- subset(rlogcounts, rownames(rlogcounts) %in% filtered_res[,1])
filtered_res_counts = t(scale(t(filtered_res_counts)))
Heatmap(filtered_res_counts[1:50,], name = "Z-Score", row_names_gp = gpar(fontsize = 8))

## Conclusion

In this tutorial, we covered the following key concepts and workflow steps:

- **Bulk RNA-seq Differer**: Downloading the dataset from an AWS S3 bucket and setting up directories
- **Quality Control**: Use FastQC and MultiQC to assess the quality of reads in the dataset and combine results for all samples to generate a comprehensive overview of quality metrics across multiple samples.
- **Adapter Trimming**: Learn how to use Trimmomatic to remove adapter sequences and low-quality bases from FASTQ reads.
- **Read Mapping and Quantification**: Understand the purpose of indexing and learn how to use STAR to create an index of the reference genome for efficient read mapping. Map reads to reference genome and quantify gene expression levels using RSEM.

In summary, this Jupyter Notebook provided a hands-on demonstration of a basic Bulk RNA-Seq analysis workflow, guiding users through the process of taking read count data, generating a list of differentially expressed genes, and creating plots. This workflow utilizes R/DESeq2 for differential gene expression analysis, which can then be further analyzed using NetAct for transcription factor network analysis in Tutorial 2b.

<div class="alert alert-block alert-warning"> NOTE: Remember to shut down your instance once you are finished with Tutorial 2.</div>